In [2]:
def print_bits(lst: list, numcols=8):
    for i in range(0, len(lst)):
        if i%numcols != numcols-1 or i == 0:
            print(lst[i], end=" ")
        else:
            print(lst[i])
    print()
        

In [3]:
def return_round_constants() -> list:
    k = []
    k.extend("0x428a2f98 0x71374491 0xb5c0fbcf 0xe9b5dba5 0x3956c25b 0x59f111f1 0x923f82a4 0xab1c5ed5".split())
    k.extend("0xd807aa98 0x12835b01 0x243185be 0x550c7dc3 0x72be5d74 0x80deb1fe 0x9bdc06a7 0xc19bf174".split())
    k.extend("0xe49b69c1 0xefbe4786 0x0fc19dc6 0x240ca1cc 0x2de92c6f 0x4a7484aa 0x5cb0a9dc 0x76f988da".split())
    k.extend("0x983e5152 0xa831c66d 0xb00327c8 0xbf597fc7 0xc6e00bf3 0xd5a79147 0x06ca6351 0x14292967".split())
    k.extend("0x27b70a85 0x2e1b2138 0x4d2c6dfc 0x53380d13 0x650a7354 0x766a0abb 0x81c2c92e 0x92722c85".split())
    k.extend("0xa2bfe8a1 0xa81a664b 0xc24b8b70 0xc76c51a3 0xd192e819 0xd6990624 0xf40e3585 0x106aa070".split())
    k.extend("0x19a4c116 0x1e376c08 0x2748774c 0x34b0bcb5 0x391c0cb3 0x4ed8aa4a 0x5b9cca4f 0x682e6ff3".split())
    k.extend("0x748f82ee 0x78a5636f 0x84c87814 0x8cc70208 0x90befffa 0xa4506ceb 0xbef9a3f7 0xc67178f2".split())
    return k

In [4]:
def shift_right(bits, num_shift: int):
    return bits >> num_shift
    '''bits = '0'* num_shift + bits
    bits = bits[0:len(bits)-num_shift]
    return bits'''

for i in range(8):
    print(bin(shift_right(0b10100111, i)))

0b10100111
0b1010011
0b101001
0b10100
0b1010
0b101
0b10
0b1


In [5]:
def rotate_right(bits, num_rotate: int, num_bits: int):
    return (2**num_bits-1)&(bits>>num_rotate|bits<<(num_bits-num_rotate))#(bits >> num_rotate)|(bits << (num_bits - num_rotate)) & 0xFFFFFFFF
    #return bits[len(bits)-num_rotate:] + bits[:len(bits)-num_rotate]

for i in range(9):
    print(bin(rotate_right(0b10100011, i, 8)))

0b10100011
0b11010001
0b11101000
0b1110100
0b111010
0b11101
0b10001110
0b1000111
0b10100011


In [70]:
def sha256(entry: str):
    #converts to binary string
    #entry = list(map(bin,bytearray(entry, encoding='utf8')))
    entry = [format(ord(i), '08b') for i in entry]
    entry = "".join(entry)
    
    #pre-processes binary input
    end = bin(len(entry))[2:]
    entry += "1"
    entry += "0"*(512-len(entry)-len(end))
    entry += end
    entry = [entry[x:x+8] for x in range(0, len(entry), 8)]
    print_bits(entry)
    
    #initialize hash values
    h0 = 0x6a09e667
    h1 = 0xbb67ae85
    h2 = 0x3c6ef372
    h3 = 0xa54ff53a
    h4 = 0x510e527f
    h5 = 0x9b05688c
    h6 = 0x1f83d9ab
    h7 = 0x5be0cd19
    
    #initialize round constants
    k = return_round_constants()
    
    #create message schedule
    message = ["".join(entry[i:i+4]) for i in range(0, len(entry), 4)]
    message.extend(["0"*32]*48)
    for i in range(len(message)):
        message[i] = int(message[i], 2)
    #print_bits(message, 2)
    
    for i in range(16, len(message)):
        s0 = rotate_right(message[i-15], 7, 32) ^ rotate_right(message[i-15], 18, 32) ^ shift_right(message[i-15], 3)
        s1 = rotate_right(message[i-2], 17, 32) ^ rotate_right(message[i-2], 19, 32) ^ shift_right(message[i-2], 10)
        message[i] = bitwise_modulo(message[i-16] + s0 + message[i-7] + s1, 32)
    
    print_bits([bin(n) for n in message], 2)
    
sha256("hello world")

01101000 01100101 01101100 01101100 01101111 00100000 01110111 01101111
01110010 01101100 01100100 10000000 00000000 00000000 00000000 00000000
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
00000000 00000000 00000000 00000000 00000000 00000000 00000000 01011000

0b1101000011001010110110001101100 0b1101111001000000111011101101111
0b1110010011011000110010010000000 0b0
0b0 0b0
0b0 0b0
0b0 0b0
0b0 0b0
0b0 0b0
0b0 0b1011000
0b110111010001110000001000110111 0b110110100001100000000110001
0b1010011101111010001000100001011 0b1111000000111110001011110000010
0b101010101100000010110011101101 0b1101001001011110111010011001001
0b10011111000011001110001011101 0b1100001011101101010010100110
0b

In [88]:
def bitwise_modulo(bits, mod_num):
    while len(bin(bits))-2 > mod_num:
        bits -= 2**(len(bin(bits))-3)
    return bits
w0 = 0b01101000011001010110110001101100
w9 = 0b0
s0 = 0b11001110111000011001010111001011
s1 = 0b0
print(bin(w0 + s0 + w9 + s1))
print(bin(bitwise_modulo(0b11011100010000100001000101001100011101, 32)))
print(bin(0b00010000100001000101001100011101))
print(len(bin(0b00110111010001110000001000110111)))
print(len(bin(0b00011)))

0b100110111010001110000001000110111
0b10000100001000101001100011101
0b10000100001000101001100011101
32
4


In [28]:
first = 0b01010101
second = 0b11110000
print(bin(first), bin(second))
res = (first<<4) | (second>>4)
print(bin(res))
print(len(str(bin(0b00000000))))
print()
x = "0b10000101000101000101111100110001"
print(bin(int(x, 2)))

0b100110111010001110000001000110111
0b110111010001110000001000110111
0b1010101 0b11110000
0b10101011111
3

0b10000101000101000101111100110001
